# Random Guess on Test Data

In [1]:
import findspark
findspark.init("/usr/spark-2.4.1")
import pyspark
from pyspark import SQLContext, SparkConf

from pyspark.sql.functions import rand,when


conf = SparkConf().setAppName("RecSys Challenge")
conf = (conf.set("deploy-mode","local")
       .set("spark.driver.memory","16g")
       .set("spark.executor.memory","16gb")
       .set("spark.driver.cores","1")
       .set("spark.num.executors","1")
       .set("spark.executor.cores","1")
       .set("spark.driver.maxResultSize", "100g"))

sc = pyspark.SparkContext()
sql = SQLContext(sc)

In [2]:
datafile = "data/val.tsv"

df = (sql.read
    .format("csv")
    .option("header", "false")
    .option("sep", "\x01")
    .load(datafile,  inferSchema="true")
    .toDF("text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains","tweet_type", "language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count","engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "engaging_user_follower_count", "engaging_user_following_count", "engaging_user_is_verified","engaging_user_account_creation", "engaged_follows_engaging"))

In [3]:
df = df.select("tweet_id", "engaging_user_id").withColumnRenamed("tweet_id", "Tweet_Id").withColumnRenamed("engaging_user_id", "User_Id")

In [4]:
df = df.withColumn('Prediction',rand())

In [5]:
df.take(1)

[Row(Tweet_Id='7647B4E9DAF4C1D8973397DC2A04F3E3', User_Id='0000006C3074607050F1339DDCB890BB', Prediction=0.14251597257530868)]

In [6]:
df.coalesce(1).write.option("header", "false").csv("val_pred")

In [7]:
df.count()

12434735